In [2]:
%pip install beautifulsoup4
%pip install langdetect
%pip install numpy
%pip install nltk
%pip install pandas
%pip install spacy
%pip install lxml
%pip install tqdm
%pip install swifter
%pip install ipywidgets
!.pyvenv/bin/python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
fish: Unknown command: .pyvenv/bin/python
fish: 
.pyvenv/bin/python -m spacy download en_core_web_sm
^~~~~~~~~~~~~~~~~^


In [3]:
# Import Python libraries
import os
import time
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import EnglishStemmer
import spacy

from tqdm.notebook import tqdm

tqdm.pandas()

In [4]:
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /home/clement/nltk_data...
[nltk_data]    |   Package movie_reviews is already

True

In [6]:
data = pd.read_csv('data/monthly_best_1000/combined.csv')

data.set_index('Id', inplace=True)
data['CreationDate'] = pd.to_datetime(data['CreationDate'])
#data['ClosedDate'] = pd.to_datetime(data['ClosedDate'])



display(data.head(3))
display(data.info())
display(data.describe())
data = data.drop(columns=['Score', 'CreationDate', 'Rank'])

,CreationDate,Title,Body,Tags,Score,Rank
Id,,,,,,
59623674,2020-01-07 07:05:02,How can I use optional chaining with arrays an...,<p>I'm trying to use optional chaining with an...,<javascript><arrays><typescript><function><opt...,320,1
59812009,2020-01-19 16:23:11,What is the use of PYTHONUNBUFFERED in docker ...,<p>I was watching a tutorial to dockerize my D...,<django><docker><dockerfile>,308,2
59601077,2020-01-05 14:54:15,IntelliJ: Error:java: error: release version 5...,<p>I'm using <strong>IntelliJ IDEA Ultimate 20...,<java><intellij-idea>,282,3


<class 'pandas.core.frame.DataFrame'>
Index: 56943 entries, 59623674 to 77645344
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   CreationDate  56943 non-null  datetime64[ns]
 1   Title         56943 non-null  object        
 2   Body          56943 non-null  object        
 3   Tags          56910 non-null  object        
 4   Score         56943 non-null  int64         
 5   Rank          56943 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(3)
memory usage: 3.0+ MB


None

,CreationDate,Score,Rank
count,56943,56943.000000,56943.000000
mean,2022-05-16 19:17:00.768891648,17.200780,500.000000
min,2020-01-01 04:27:54,2.000000,1.000000
25%,2021-03-08 16:25:38,6.000000,250.000000
50%,2022-05-16 14:24:36,11.000000,500.000000
75%,2023-07-24 05:58:01,19.000000,750.000000
max,2024-09-21 19:38:11,2427.000000,999.000000
std,NaN,29.363405,288.388847


In [7]:
def extract_code(text):
  str = '\n'.join(re.findall(r'<code>(?:\s*\w+\s+){1,}\w+\s*</code>', text))
  return BeautifulSoup(str, 'lxml').get_text()

In [6]:

#data['Code'] = data['Body'].progress_apply(extract_code)

In [7]:
#data['BodyWithoutCode'] = data['Body'].progress_apply(lambda x: re.sub(r'<code>(?:\s*\w+\s+){1,}\w+\s*</code>', '', x))


In [8]:
data['BodyWithoutCode'] = data['Body'].progress_apply(lambda x: BeautifulSoup(x, 'lxml').get_text())

  0%|          | 0/56943 [00:00<?, ?it/s]

In [9]:

data['Concatenated'] = data['Title'] + '. ' + data['BodyWithoutCode']
data['Concatenated'] = data['Concatenated'].str.lower()

data = data.drop(columns=['Body', 'Title', 'BodyWithoutCode'])

In [10]:
def is_english(text):
    try:
        return detect(text[0:100]) == 'en'
    except:
        return False


data = data[data['Concatenated'].progress_apply(is_english)]

  0%|          | 0/56943 [00:00<?, ?it/s]

In [11]:
nlp = spacy.load('en_core_web_sm')
stop_words = stopwords.words("english")
wn = WordNetLemmatizer()

In [1]:
def filter_tokens(token):
   return token.pos_ in ["NOUN", "PROPN"] and token.text.lower() not in stop_words


def preprocess(text):
   normalized_text = text
   # Tokeniser le texte
   text = re.sub(r'c#', 'csharp', text)
   text = re.sub(r'c\+\+', 'cplusplus', text)
   text = re.sub(r'\.net', 'dotnet', text)
   doc = nlp(text)

   filtered = list(filter(filter_tokens, doc))
   lemmatized = list(map(lambda token: token.lemma_, filtered))


   # join lemmatized tokens
   return ' '.join(lemmatized)


data['Processed'] = data['Concatenated'].progress_apply(preprocess)
data = data.drop(columns=['Concatenated'])


NameError: name 'data' is not defined

In [14]:

data['Tags'] = (data['Tags'].str.lstrip('<').str.rstrip('>').str.split('><')).str.join(',')
# Export to CSV
data.to_csv("data/StackOverflow_tokenized2.csv", sep=";")